# Double Gyre

In [1]:
import os
import shutil
import yaml
import numpy as np

### Parameters we want altered :

In MOM_override
For P2L 
- NIGLOBAL (220, 110, 55, 27) (total lenx = 2200)
- NJGLOBAL (200, 100, 50, 25) (total leny = 2000)
- thickness_flux_model_type ("nondim_ANN" or "GM ann")
- thickness_flux_ann_num_layers (4 or 2)
- thickness_flux_ann_params_file ("INPUT/main_ANN_window_3.nc" or "INPUT/thickness_diffuse_k1000_ann_params.nc")
- thickness_flux_ann_coeff (0 -> 1)


In mom.sub 
- #SBATCH --job-name="MOM6" to #SBATCH --job-name="<whatever is right name here.>"

In [2]:
res = [10, 20, 40, 80]
C_ANN = np.array([0.0, 0.25, 0.5, 0.75, 1.0])
C_GM = np.array([10, 50, 100, 200, 500])/1000
model_types = ['ANN', 'GM1000']

In [3]:
# Generate exp P2L 

Lx = 2200
Ly = 2000

exp_dic = {}
for ANN_type in model_types:
    for r in res: 
        NIGLOBAL = int(Lx/r)
        NJGLOBAL = int(Ly/r)
        
        if ANN_type == 'ANN': 
            C = C_ANN
            model_type = 'nondim_ann'
            num_layers = 4
            param_file = "INPUT/main_ANN_window_3.nc"
            
        elif ANN_type == 'GM1000':
            C = C_GM 
            model_type = 'GM_ann'
            num_layers = 2
            param_file = "INPUT/thickness_diffuse_k1000_ann_params.nc"
        
        for coeff in C: 
    
            exp_name = 'res_' + str(r) + 'km_' + str(ANN_type) + '_' + str(coeff)
            # print(exp_name)
            
            exp_dic[exp_name] = {}
            
            exp_dic[exp_name]['MOM_override_params'] = {'NIGLOBAL': NIGLOBAL, 
                                                        'NJGLOBAL': NJGLOBAL,
                                                        'DAYMAX' : 14400.,
                                                        'thickness_flux_model_type': model_type,
                                                        'thickness_flux_ann_num_layers': num_layers,
                                                        'thickness_flux_ann_params_file': param_file,
                                                        'thickness_flux_ann_coeff': coeff} 

            exp_dic[exp_name]['mom.sub_params'] = {'--job-name': exp_name,
                                                   '--time': '03:00:00'} 

            if r>40: 
                exp_dic[exp_name]['mom.sub_params'] = {'--job-name': exp_name,
                                                      '--time': '01:00:00', 
                                                      '--ntasks-per-node': 16} 
            if r<20:
                exp_dic[exp_name]['mom.sub_params'] = {'--job-name': exp_name,
                                                      '--time': '09:00:00', 
                                                      '--ntasks-per-node': 48}

            exp_dic[exp_name]['input.nml_params'] = {} 

In [4]:
def to_serializable(obj):
    if isinstance(obj, dict):
        return {k: to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [to_serializable(v) for v in obj]
    elif isinstance(obj, tuple):
        return tuple(to_serializable(v) for v in obj)
    elif isinstance(obj, np.generic):
        return obj.item()  # convert numpy scalar to Python scalar
    else:
        return obj

In [5]:
with open("experiments_DG_40year.yaml", "w") as f:
    yaml.dump({"experiments": to_serializable(exp_dic)}, f, default_flow_style=False)


## Code that was used to create the experiment runner. 

In [18]:
def load_config(config_file="experiments.yaml"):
     with open(config_file, "r") as f:
         return yaml.safe_load(f)["experiments"]

In [19]:
load_config()

{'res_10km_ANN_0.0': {'MOM_override_params': {'DAYMAX': 1.0,
   'NIGLOBAL': 120,
   'NJGLOBAL': 160,
   'thickness_flux_ann_coeff': 0.0,
   'thickness_flux_ann_num_layers': 4,
   'thickness_flux_ann_params_file': 'INPUT/main_ANN_window_3.nc',
   'thickness_flux_model_type': 'nondim_ANN'},
  'input.nml_params': {},
  'mom.sub_params': {'--job-name': 'res_10km_ANN_0.0', '--time': '00:03:00'}},
 'res_10km_ANN_0.25': {'MOM_override_params': {'DAYMAX': 1.0,
   'NIGLOBAL': 120,
   'NJGLOBAL': 160,
   'thickness_flux_ann_coeff': 0.25,
   'thickness_flux_ann_num_layers': 4,
   'thickness_flux_ann_params_file': 'INPUT/main_ANN_window_3.nc',
   'thickness_flux_model_type': 'nondim_ANN'},
  'input.nml_params': {},
  'mom.sub_params': {'--job-name': 'res_10km_ANN_0.25', '--time': '00:03:00'}},
 'res_10km_ANN_0.5': {'MOM_override_params': {'DAYMAX': 1.0,
   'NIGLOBAL': 120,
   'NJGLOBAL': 160,
   'thickness_flux_ann_coeff': 0.5,
   'thickness_flux_ann_num_layers': 4,
   'thickness_flux_ann_params_f

In [20]:
import time

def backup_run_dir(src_dir, backup_root, exp_name):
    """
    Copy the existing run directory to a backup location with a timestamp.
    """
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    backup_dir = os.path.join(backup_root, f"{exp_name}_{timestamp}")
    shutil.copytree(src_dir, backup_dir)
    print(f"[💾] Backed up {exp_name} to {backup_dir}")

In [21]:
def merge_kv_lines(
    base_text,
    overrides,
    prefix="",
    assign_op="=",
    comment_prefix=None,
    quote_keys=None,
):
    """
    General key=value merger with optional smart quoting.
    
    Parameters:
        base_text       : str (original file contents)
        overrides       : dict (key: value pairs to override or add)
        prefix          : str (text before the key, e.g. '#override ')
        assign_op       : str (e.g. '=', ':')
        comment_prefix  : str or None — restricts which lines to consider
        quote_keys      : set of keys — only these keys will be quoted if values are strings

    Returns:
        str : modified file content
    """

    def format_value(key, val):
        if isinstance(val, str):
            if (quote_keys is None or key in quote_keys):
                if not (val.startswith('"') and val.endswith('"')):
                    return f'"{val}"'
        return val

    lines = base_text.splitlines()
    updated = []
    keys = set(overrides.keys())

    for line in lines:
        stripped = line.strip()
        use_line = True

        if comment_prefix is None or stripped.startswith(comment_prefix):
            if prefix in stripped and assign_op in stripped:
                content = stripped.replace(prefix, "", 1).strip()
                key = content.split(assign_op, 1)[0].strip()

                if key in overrides:
                    val = format_value(key, overrides[key])
                    updated.append(f"{prefix}{key} {assign_op} {val}")
                    keys.remove(key)
                    use_line = False

        if use_line:
            updated.append(line)

    for key in keys:
        val = format_value(key, overrides[key])
        updated.append(f"{prefix}{key} {assign_op} {val}")

    return "\n".join(updated)


In [22]:
def prepare_experiment(
    name,
    config,
    mode="fresh",
    update_submission_script_flag=False,
    update_input_nml_flag=False,
    backup_root=None 
):
    run_dir = os.path.join(RUNS_DIR, name)
    override_path = os.path.join(run_dir, "MOM_override")
    momsub_path = os.path.join(run_dir, "mom.sub")
    nml_path = os.path.join(run_dir, "input.nml")

    # Handle run directory creation/update
    if os.path.exists(run_dir):
        if mode == "skip":
            print(f"[⏭] Skipping existing: {name}")
            return
        elif mode == "fresh":
            shutil.rmtree(run_dir)
            shutil.copytree(BASE_DIR, run_dir)
        elif mode == "update":
            # 🔒 Safety: backup before update
            if backup_root is not None:
                os.makedirs(backup_root, exist_ok=True)
                backup_run_dir(run_dir, backup_root, name)
            # Don't modify files just yet — rest of logic follows
        else:
            raise ValueError(f"Unknown mode: {mode}")
    else:
        shutil.copytree(BASE_DIR, run_dir)

    # === MOM_override ===
    mom_override_cfg = config.get("MOM_override_params", {})
    if os.path.exists(override_path):
        with open(override_path, "r") as f:
            base_override = f.read()
    else:
        base_override = ""
    override_text = merge_kv_lines(base_override, mom_override_cfg, prefix="#override ", assign_op="=", comment_prefix="#override")
    with open(override_path, "w") as f:
        f.write("! Auto-generated MOM_override\n" + override_text)

    # === mom.sub ===
    if update_submission_script_flag:
        mom_sub_cfg = config.get("mom.sub_params", {})
        if os.path.exists(momsub_path):
            with open(momsub_path, "r") as f:
                base_sub = f.read()
        else:
            base_sub = ""
        sub_text = merge_kv_lines(base_sub, mom_sub_cfg, prefix="#SBATCH ", assign_op="=", comment_prefix="#SBATCH", quote_keys={"--job-name"})
        with open(momsub_path, "w") as f:
            f.write(sub_text)

    # === input.nml ===
    if update_input_nml_flag and "input.nml_params" in config:
        nml_cfg = config["input.nml_params"]
        if os.path.exists(nml_path):
            with open(nml_path, "r") as f:
                base_nml = f.read()
        else:
            base_nml = ""
        nml_text = merge_kv_lines(base_nml, nml_cfg, prefix="", assign_op="=", comment_prefix=None)
        with open(nml_path, "w") as f:
            f.write(nml_text)

    print(f"[✓] Prepared ({mode}): {name}")


In [23]:
def generate_all_experiments(mode="fresh", 
                            backup_root=None,
                            update_submission_script_flag=False, 
                            update_input_nml_flag=False,
                            ):
    
    experiments = load_config()
    os.makedirs(RUNS_DIR, exist_ok=True)
    
    for name, config in experiments.items():
        prepare_experiment(
            name,
            config,
            mode=mode,
            update_submission_script_flag=update_submission_script_flag,
            update_input_nml_flag=update_input_nml_flag,
            backup_root = backup_root
        )


In [24]:
generate_all_experiments(update_submission_script_flag=True)

[✓] Prepared (fresh): res_10km_ANN_0.0
[✓] Prepared (fresh): res_10km_ANN_0.25
[✓] Prepared (fresh): res_10km_ANN_0.5
[✓] Prepared (fresh): res_10km_ANN_0.75
[✓] Prepared (fresh): res_10km_ANN_1.0
[✓] Prepared (fresh): res_10km_GM1000_0.01
[✓] Prepared (fresh): res_10km_GM1000_0.05
[✓] Prepared (fresh): res_10km_GM1000_0.1
[✓] Prepared (fresh): res_10km_GM1000_0.5
[✓] Prepared (fresh): res_10km_GM1000_1.0
[✓] Prepared (fresh): res_20km_ANN_0.0
[✓] Prepared (fresh): res_20km_ANN_0.25
[✓] Prepared (fresh): res_20km_ANN_0.5
[✓] Prepared (fresh): res_20km_ANN_0.75
[✓] Prepared (fresh): res_20km_ANN_1.0
[✓] Prepared (fresh): res_20km_GM1000_0.01
[✓] Prepared (fresh): res_20km_GM1000_0.05
[✓] Prepared (fresh): res_20km_GM1000_0.1
[✓] Prepared (fresh): res_20km_GM1000_0.5
[✓] Prepared (fresh): res_20km_GM1000_1.0
[✓] Prepared (fresh): res_40km_ANN_0.0
[✓] Prepared (fresh): res_40km_ANN_0.25
[✓] Prepared (fresh): res_40km_ANN_0.5
[✓] Prepared (fresh): res_40km_ANN_0.75
[✓] Prepared (fresh): re

In [25]:
import subprocess
import csv
from datetime import datetime
import os

def submit_and_log_job(run_dir, exp_name, log_path="submitted_jobs.csv"):
    result = subprocess.run(
        ["sbatch", "mom.sub"],
        cwd=run_dir,
        capture_output=True,
        text=True
    )

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    if result.returncode == 0:
        output = result.stdout.strip()
        print(f"[🚀] Submitted {exp_name}: {output}")

        if "Submitted batch job" in output:
            job_id = output.split()[-1]

            # Append to CSV log
            with open(log_path, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([timestamp, exp_name, job_id, run_dir])

            return job_id
    else:
        print(f"[❌] Submission failed for {exp_name}: {result.stderr.strip()}")
        return None


In [26]:
def run_all_experiments_with_tracking(runs_root="runs", log_path="submitted_jobs.csv"):
    # If the log doesn't exist yet, create it with a header
    if not os.path.exists(log_path):
        with open(log_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["timestamp", "experiment", "job_id", "path"])

    for exp_name in sorted(os.listdir(runs_root)):
        run_dir = os.path.join(runs_root, exp_name)
        momsub_path = os.path.join(run_dir, "mom.sub")

        if not os.path.isdir(run_dir) or not os.path.exists(momsub_path):
            print(f"[⚠] Skipping {exp_name} — no mom.sub found.")
            continue

        submit_and_log_job(run_dir, exp_name, log_path=log_path)


In [27]:
import subprocess
import csv

def cancel_tracked_jobs(log_path="submitted_jobs.csv", dry_run=False):
    """
    Cancel all jobs listed in the log file that are still in the SLURM queue.
    
    Parameters:
        log_path : str — path to the job tracking CSV
        dry_run  : bool — if True, don't actually cancel jobs, just print them
    """
    if not os.path.exists(log_path):
        print("[⚠] No job log found.")
        return

    # Step 1: Load logged job IDs
    with open(log_path, "r") as f:
        reader = csv.DictReader(f)
        jobs = [row for row in reader]

    if not jobs:
        print("[ℹ] No jobs to cancel.")
        return

    # Step 2: Get list of currently active jobs
    try:
        result = subprocess.run(["squeue", "-h", "-o", "%A"], capture_output=True, text=True, check=True)
        active_job_ids = set(result.stdout.strip().splitlines())
    except subprocess.CalledProcessError as e:
        print("[❌] Failed to check running jobs:", e)
        return

    # Step 3: Cancel jobs that are still running
    canceled = 0
    for job in jobs:
        job_id = job["job_id"]
        if job_id in active_job_ids:
            if dry_run:
                print(f"[DRY-RUN] Would cancel job {job_id} ({job['experiment']})")
            else:
                print(f"[🛑] Canceling job {job_id} ({job['experiment']})...")
                try:
                    subprocess.run(["scancel", job_id], check=True)
                    canceled += 1
                except subprocess.CalledProcessError as e:
                    print(f"[⚠] Failed to cancel job {job_id}: {e}")
        else:
            print(f"[✓] Job {job_id} ({job['experiment']}) is no longer running.")

    print(f"[✅] Done. {canceled} job(s) canceled.")


In [29]:
run_all_experiments_with_tracking(runs_root=RUNS_DIR)

FileNotFoundError: [Errno 2] No such file or directory: 'sbatch'

In [ ]:
FileNotFoundError: [Errno 2] No such file or directory: 'sbatch'